<a href="https://colab.research.google.com/github/0ethel0zhang/yoga_training/blob/main/Yoga_Courses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [ ]:
import re
from collections import Counter

In [ ]:
# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = '14SuV94KdgsHzOIhZBFcjJioqvFOADi72NKWsL6t3nHE'
SAMPLE_RANGE_NAME = '107_poses!A1:F'
SHEET_NAME = "Yoga Poses"

In [ ]:
#@title Import from Google Sheets
# (https://colab.research.google.com/notebooks/snippets/sheets.ipynb#scrollTo=JiJVCmu3dhFa)
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

worksheet = gc.open(SHEET_NAME).sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
# print(rows)

# Convert to a DataFrame and render.
df = pd.DataFrame.from_records(rows[1:], columns = rows[0])

# Data Cleaning

In [ ]:
df.shape

(111, 11)

In [ ]:
df.columns

Index(['Sanskrit', 'English', 'Anatomic Benefits', 'Energetic Benefits',
       'Possible Injury', 'Modification', 'Alignment Issues',
       'Strengths Required', 'Flexibility Required', 'Simpler Poses',
       'Instruction'],
      dtype='object')

In [ ]:
# all matching
# df["Hindi"].head(50).apply(lambda x: x[0] == x[1] == x[2], axis = 1).sum()
# df[df["Pose"] != df["Hindi"].str.lower()][["Pose", "Hindi"]]

In [ ]:
def find_poses_of_x_benefits(x, col="Anatomic Benefits", df=df):
  return df[df[col].str.lower().str.contains(x)]

In [ ]:
find_poses_of_x_benefits("abdomin")[simple_cols]

,Sanskrit,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
1,Janu Sirsasana,Head-to-Knee Forward Bend,Sit with one leg extended and the other foot a...,"Stretches hamstrings, opens hips, and stimulat...",Calms the mind and relieves stress,Seated Forward Bend Pose,Head-to-Knee Forward Bend Pose
2,Chaturanga Dandasana,Four-Limbed Staff Pose,"Start in plank position, lower the body down w...","Strengthens arms, wrists, and core; tones abdo...",Builds heat and enhances endurance,Plank Pose or Modified Push-up Pose,Four-Limbed Staff Pose
20,Baddha Konasana,Bound Angle Pose,"Sit with feet together, knees out to the sides...",Opens hips and groin; stimulates abdominal organs,Connects with the root chakra; promotes a sens...,Butterfly Pose,Bound Angle Pose
108,Tri Pada Adho Mukha Svanasana,Three legged downward facing dog,1. Start in a Downward-Facing Dog. 2. Keep bot...,"Stretches the lower body, hamstrings, calves, ...","Stimulates your blood flow, as your heart is a...","Downward-Facing Dog (Adho Mukha Svanasana), Pl...",Three legged downward facing dog Pose


In [ ]:
def find_poses_by_name(x, df=df):
  return df[df["English"].str.lower().str.contains(x.lower())]

In [ ]:
out_df = find_poses_of_x_benefits("abdomin")
for p in ["head", "digestion"]:
  out_df = pd.concat([out_df, find_poses_of_x_benefits(p)], axis = 0)
out_df[["English","Instruction"]]

,English,Instruction
1,Head-to-Knee Forward Bend,Sit with one leg extended and the other foot a...
2,Four-Limbed Staff Pose,"Start in plank position, lower the body down w..."
20,Bound Angle Pose,"Sit with feet together, knees out to the sides..."
108,Three legged downward facing dog,1. Start in a Downward-Facing Dog. 2. Keep bot...
9,Thunderbolt Pose,Kneel with the buttocks on the heels and the h...


In [ ]:
find_poses_by_name("lunge")

,Sanskrit,English,Anatomic Benefits,Energetic Benefits,Possible Injury,Modification,Alignment Issues,Strengths Required,Flexibility Required,Simpler Poses,Instruction
41,Utthita Ashwa Sanchalanasana,Extended Low Lunge,Strengthens legs and core; stretches hip flexors,Builds heat and energy in the body; enhances d...,Strain on hips and lower back; risk of falling,"Warm up hips, engage core, and use props for s...",Knee and hip alignment; high lunge,Leg and hip strength; ankle stability,N/A,High Lunge Pose or Crescent Lunge Pose,"From a low lunge position, lift the torso, rea..."
94,Anjaneyasana,Low Lunge Pose,Stretches hip flexors and quads; opens chest a...,Activates energy in the solar plexus; enhances...,Hip and knee strain; discomfort for individual...,"Modify for hip and knee comfort, engage core, ...",Knee and hip alignment; low lunge pose,Leg and hip strength; hip flexibility,N/A,Low Lunge Pose or Crescent Lunge Pose,"From a low lunge position, lower the hips, lif..."


In [ ]:
body_parts = ["leg", "hip", "shoulder", "chest", "neck", "back", "arm",
              "front", "core", "hamstring", "ankle", "wrist", "foot", "knee", "hand", "head"]

In [ ]:
def add_end_pose(p):
  return p if "Pose" in p else p+" Pose"

In [ ]:
df["Pose"] = df.English.apply(add_end_pose)

In [ ]:
def get_rid_of_repetition(x):
  text = " or ".join([add_end_pose(m) if m != " or " else m for m in x["Simpler Poses"].split(" or ")])
  s = add_end_pose(x["English"])
  try:
    t = re.findall("( )*(or)*( )*("+s+")( )*(or)*( )*", text)[0]
  except:
    return text
    pass
  if Counter(t)["or"] > 1:
    return text.replace("".join(t), " or ")
  else:
    return text.replace("".join(t), "")

In [ ]:
df["Intro Poses"] = df.apply(get_rid_of_repetition, axis = 1)
# df[df["Intro Poses"] != df["Simpler Poses"]] #check if updated correctly

In [ ]:
simple_cols = ['Sanskrit', 'English', 'Instruction', 'Anatomic Benefits', 'Energetic Benefits',
 'Intro Poses', "Pose"]

In [ ]:
pd.concat([find_poses_of_x_benefits("shoulder"),
           find_poses_of_x_benefits("back"),
           find_poses_of_x_benefits("neck")], axis = 0)[simple_cols].drop_duplicates()

,Sanskrit,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
3,Ashtanga Namaskara,Eight-Limbed Salutation,"From plank, lower the knees, chest, and chin t...","Strengthens arms, shoulders, and chest; tones ...",Builds strength and prepares the body for back...,Knees-Chest-Chin Pose or Modified Chaturanga Pose,Eight-Limbed Salutation Pose
4,Camatkarasana,Wild Thing,"From Downward-Facing Dog, lift one leg, bend t...",Opens chest and shoulders; strengthens arms,Releases tension and promotes a sense of joy,Bridge Pose,Wild Thing Pose
5,Gomukhasana,Cow Face Pose,"Sit with legs crossed, stack one knee on top o...",Stretches shoulders and hips; improves posture,Balances the nervous system and enhances conce...,Eagle Arms Pose,Cow Face Pose
6,Makarasana,Crocodile Pose,"Lie on the stomach, place hands under the fore...",Relaxes the back and shoulders; relieves stress,Encourages deep relaxation and rejuvenation,Sphinx Pose,Crocodile Pose
11,Vrischikasana,Scorpion Pose,"Lie on the stomach, place hands beside the che...",Strengthens arms and back; opens chest and sho...,Activates energy in the lower chakras and stim...,Sphinx Pose or Cobra Pose,Scorpion Pose
...,...,...,...,...,...,...,...
105,Paschimottanasana,Seated Forward Bend,"Sit with legs extended, hinge at the hips, and...",Stretches hamstrings and lower back; improves ...,Calms the mind and releases tension,Staff Pose,Seated Forward Bend Pose
43,Matsyasana,Fish Pose,"Lie on the back, place the hands under the hip...",Opens chest and throat; stretches neck and spine,Stimulates the throat chakra; promotes self-ex...,Supported Fish Pose,Fish Pose
55,Balasana,Child's Pose,"Kneel on the floor, separate your knees about ...",Relaxes the spine and neck; reduces stress,Promotes a sense of surrender and relaxation,,Child's Pose
69,Bitilasana,Cow Pose,"Start on hands and knees, arch the back down, ...",Stretches spine and neck; improves flexibility,Activates the heart chakra; promotes emotional...,Cat-Cat Pose,Cow Pose


In [ ]:
find_poses_of_x_benefits(body_parts[-3])[simple_cols]

,Sanskrit,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose


# Course plans

## Final Project

In [ ]:
final_project = """Mountain pose
Chair
Warrior I
Low lunge
Lizard
Twisted lizard
Warrior I
Mountain pose
Forward fold
Rag-doll rolls
Chair
Warrior I
Low lunge
Lizard
Twisted lizard
Warrior I
Chair
Mountain
Forward fold
Halfway bend
Forward fold
Plank
Downward dog
Three legged dog
Three legged dog bent knee variation
Pigeon
Three legged dog
Downward dog
Three legged dog
Three legged dog bent knee variation
Pigeon
Three legged dog
Downward dog
Plank
Chaturanga dandasana
Upward-Facing Dog Pose
Downward dog
Forward fold
Halfway bend
Forward fold
Mountain""".replace("Forward fold", "Standing Forward Bend").replace(
    "Halfway bend", "Standing Half Forward Bend").replace(
    "Downward dog", "Downward-Facing Dog Pose").replace(
    "Chaturanga dandasana", "Four-Limbed Staff Pose").replace(
    "Three legged dog", "Three legged downward facing dog"
    ).split("\n")

In [ ]:
final_project_df = pd.DataFrame(final_project, columns = ["English"])
final_project_df["Sanskrit"] = final_project_df.English.apply(lambda x: find_poses_by_name(x).Sanskrit.values[0]
                               if len(find_poses_by_name(x).Sanskrit) > 0 else "")
final_project_df = final_project_df.merge(df.drop(columns = ["English"]), on = "Sanskrit", how = "left")

In [ ]:
from google.colab import data_table
data_table.DataTable(
    final_project_df,
    include_index=False, num_rows_per_page=50)

,English,Sanskrit,Anatomic Benefits,Energetic Benefits,Possible Injury,Modification,Alignment Issues,Strengths Required,Flexibility Required,Simpler Poses,Instruction,Pose,Intro Poses
0,Mountain pose,Tadasana,Improves posture and alignment; strengthens legs,Grounds energy and improves focus; enhances st...,Poor alignment leading to back pain,"Focus on proper posture, engage core, and dist...",Alignment of spine and pelvis; standing pose,Core strength; ankle stability,N/A,Mountain Pose,"Stand with feet together, arms by the sides, a...",Mountain Pose,
1,Chair,Utkatasana,Strengthens legs and core; opens chest and sho...,Builds heat and energy in the body; enhances d...,Knee and hip strain; discomfort for individual...,"Modify for comfort, use props, and engage core",Knee and hip alignment; chair pose,Leg and hip strength; ankle stability,N/A,Chair Pose or Goddess Pose,"From Mountain Pose, bend the knees, sit back a...",Chair Pose,Goddess Pose
2,Warrior I,Virabhadrasana I,"Strengthens legs, opens hips and chest",Energizes and focuses the mind,Knee and hip strain; risk of falling,"Align knee with ankle, engage core, and practi...",Front knee alignment with ankle; hip and shoul...,Leg and hip strength; shoulder and core strength,Open hips and shoulders,Warrior II Pose,"From Mountain Pose, step one foot back and ben...",Warrior I Pose,Warrior II Pose
3,Low lunge,Utthita Ashwa Sanchalanasana,Strengthens legs and core; stretches hip flexors,Builds heat and energy in the body; enhances d...,Strain on hips and lower back; risk of falling,"Warm up hips, engage core, and use props for s...",Knee and hip alignment; high lunge,Leg and hip strength; ankle stability,N/A,High Lunge Pose or Crescent Lunge Pose,"From a low lunge position, lift the torso, rea...",Extended Low Lunge Pose,High Lunge Pose or Crescent Lunge Pose
4,Lizard,Utthan Pristhasana,Opens up your chest and hips; stretches your g...,Improves reproductive health; stimulates inter...,Knee or hip strain if not performed with prope...,Use props (blocks or blankets) under the hands...,Ensure the front knee is directly over the ank...,"Quadriceps, hip flexors, core muscles.","Hip flexors, hamstrings.","Low Lunge (Anjaneyasana), Downward-Facing Dog ...",1. Start in a Downward-Facing Dog. 2. Bring th...,Lizard Pose,"Low Lunge (Anjaneyasana), Downward-Facing Dog ..."
5,Twisted lizard,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Warrior I,Virabhadrasana I,"Strengthens legs, opens hips and chest",Energizes and focuses the mind,Knee and hip strain; risk of falling,"Align knee with ankle, engage core, and practi...",Front knee alignment with ankle; hip and shoul...,Leg and hip strength; shoulder and core strength,Open hips and shoulders,Warrior II Pose,"From Mountain Pose, step one foot back and ben...",Warrior I Pose,Warrior II Pose
7,Mountain pose,Tadasana,Improves posture and alignment; strengthens legs,Grounds energy and improves focus; enhances st...,Poor alignment leading to back pain,"Focus on proper posture, engage core, and dist...",Alignment of spine and pelvis; standing pose,Core strength; ankle stability,N/A,Mountain Pose,"Stand with feet together, arms by the sides, a...",Mountain Pose,
8,Standing Forward Bend,Uttanasana,Stretches hamstrings and lower back; improves ...,Calms the mind and releases tension,Strain on lower back and hamstrings,"Bend knees, engage core, and use props for sup...",Lower back and hamstring alignment; forward fold,Core strength; hamstring flexibility,Open hamstrings and lower back,Standing Forward Bend or Forward Fold with Ben...,"Stand with feet hip-width apart, hinge at the ...",Standing Forward Bend Pose,Forward Fold with Bent Knees Pose
9,Rag-doll rolls,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Gentle Vinyasa

In [ ]:
gentle_vinyasa = '''Section 1: Flow Sequence
Extended Child's Pose
Blank Upward-Facing Dog
Extended Child's Pose
Downward-Facing Dog
Plank
Side Plank
Downward-Facing Dog
Rag Doll
Mountain
Section 2: Warrior Series
One Legged Prayer
Warrior I
Warrior II
Peaceful Warrior
Warrior II
Extended Side Angle
Warrior III
Wide Legged Forward Bend
Mountain
Chair
Rag Doll
Section 3: Cool Down and Floor Poses
Downward-Facing Dog
Easy
Head-to-Knee Forward Bend
Half Lord of the Fishes
Bridge
Happy Baby
Corpse'''.replace("Forward fold", "Standing Forward Bend").replace(
    "Halfway bend", "Standing Half Forward Bend").replace(
    "Downward dog", "Downward-Facing Dog Pose").replace(
    "Chaturanga dandasana", "Four-Limbed Staff Pose").replace(
    "Three legged dog", "Three legged downward facing dog"
    ).split("\n")

In [ ]:
gentle_vinyasa_df = pd.DataFrame(gentle_vinyasa, columns = ["English"])
gentle_vinyasa_df["Sanskrit"] = gentle_vinyasa_df.English.apply(lambda x: find_poses_by_name(x).Sanskrit.values[0]
                               if len(find_poses_by_name(x).Sanskrit) > 0 else "")
gentle_vinyasa_df = gentle_vinyasa_df.merge(df.drop(columns = ["English"]), on = "Sanskrit", how = "left")

In [ ]:
from google.colab import data_table
data_table.DataTable(
    gentle_vinyasa_df,
    include_index=False, num_rows_per_page=50)

,English,Sanskrit,Anatomic Benefits,Energetic Benefits,Possible Injury,Modification,Alignment Issues,Strengths Required,Flexibility Required,Simpler Poses,Instruction,Pose,Intro Poses
0,Section 1: Flow Sequence,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Extended Child's Pose,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Blank Upward-Facing Dog,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Extended Child's Pose,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Downward-Facing Dog,Adho Mukha Svanasana,Stretches hamstrings and calves; improves posture,Calms the mind and releases tension,"Strain on shoulders, wrists, and hamstrings; r...","Warm up shoulders and wrists, engage core, and...",Spine and hip alignment; downward-facing dog pose,Core strength; hamstring flexibility,N/A,Downward-Facing Dog Pose or Puppy Pose,"Start in Downward-Facing Dog, lifting the hips...",Downward-Facing Dog Pose,Puppy Pose
5,Plank,Purvottanasana,"Stretches shoulders, chest, and wrists; streng...",Activates the solar plexus; promotes a sense o...,"Strain on wrists, shoulders, and lower back; r...","Warm up wrists and shoulders, engage core, and...",Wrist and shoulder alignment; reverse plank,Wrist and shoulder strength; core stability,N/A,Upward Plank Pose or Reverse Tabletop Pose,"Sit with legs extended, place hands behind the...",Upward Plank Pose,Reverse Tabletop Pose
6,Side Plank,Vasisthasana,"Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,"Strain on wrists, shoulders, and lower back; r...","Warm up wrists and shoulders, engage core, and...",Wrist and shoulder alignment; side plank,Wrist and shoulder strength; core stability,N/A,Side Plank Pose or Modified Side Plank Pose,"From Plank Pose, shift weight to one hand, sta...",Side Plank Pose,Modified Side Plank Pose
7,Downward-Facing Dog,Adho Mukha Svanasana,Stretches hamstrings and calves; improves posture,Calms the mind and releases tension,"Strain on shoulders, wrists, and hamstrings; r...","Warm up shoulders and wrists, engage core, and...",Spine and hip alignment; downward-facing dog pose,Core strength; hamstring flexibility,N/A,Downward-Facing Dog Pose or Puppy Pose,"Start in Downward-Facing Dog, lifting the hips...",Downward-Facing Dog Pose,Puppy Pose
8,Rag Doll,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Mountain,Tadasana,Improves posture and alignment; strengthens legs,Grounds energy and improves focus; enhances st...,Poor alignment leading to back pain,"Focus on proper posture, engage core, and dist...",Alignment of spine and pelvis; standing pose,Core strength; ankle stability,N/A,Mountain Pose,"Stand with feet together, arms by the sides, a...",Mountain Pose,


## Gentle Flow

In [ ]:
gentle_flow = '''Easy
Seated Mountain
Salutation Seal
Seated Side Bend
Table Top
Cat
Cow
Extended Child's Pose
Downward-Facing Dog
Standing Forward Bend

Section 2: Standing Poses
Upward Forward Fold Half Salute
Standing Forward Bend
Upward Salute II
Crescent Moon
Upward Salute II
Warrior I
Warrior II
Triangle
Warrior II
Chair
Standing Forward Bend

Section 3: Seated and Forward Bends
Upward Forward Fold Half Salute
Standing Forward Bend
Upward Salute II
Crescent Moon
Upward Salute II
Warrior I
Warrior II
Triangle
Warrior II
Chair
Standing Forward Bend

Section 4: Seated and Restorative Poses
Upward Forward Fold Half Salute
Standing Forward Bend
Staff
Head-to-Knee Forward Bend
Marichi
Staff
Head-to-Knee Forward Bend
Marichi
Reclining Mountain
Legs-Up-the-Wall
Knees to Chest
Section 5: Supine and Restorative Poses
Knee Stretch
Hip Roll
Knees to Chest
Knee Stretch
Hip Roll
Knees to Chest
Happy Baby
Corpse'''.split("\n")

In [ ]:
gentle_flow_df = pd.DataFrame(gentle_flow, columns = ["English"])
gentle_flow_df["Sanskrit"] = gentle_flow_df.English.apply(lambda x:
  find_poses_of_x_benefits(x, "English").Sanskrit.values[0]
  if len(find_poses_of_x_benefits(x, "English").Sanskrit) > 0 else "")
gentle_flow_df = gentle_flow_df.merge(df.drop(columns = ["English"]),
                                      on = "Sanskrit", how = "left")

In [ ]:
from google.colab import data_table
data_table.DataTable(
    gentle_flow_df[simple_cols],
    include_index=False, num_rows_per_page=50)

## Stretch

In [ ]:
sequence_df_by_body_part_w_dup("stretch")[simple_cols]

<ipython-input-6-57fa8158c2b8>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return df[df[col].str.contains(x)]


,Sanskrit,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
0,Virabhadrasana II,Warrior II,"From Mountain Pose, step one foot back and ben...",Strengthens legs and opens hips; stretches groins,Builds strength and stability; enhances determ...,Warrior I Pose or Extended Side Angle Pose,Warrior II Pose
1,Virabhadrasana I,Warrior I,"From Mountain Pose, step one foot back and ben...","Strengthens legs, opens hips and chest",Energizes and focuses the mind,Warrior II Pose,Warrior I Pose
2,Utthita Parsvakonasana,Extended Side Angle Pose,"From Warrior II, extend the front arm forward,...","Strengthens legs, hips, and core; stretches si...",Energizes and balances the body; enhances stamina,Warrior II Pose,Extended Side Angle Pose
3,Virabhadrasana III,Warrior III,"From Mountain Pose, step one foot back, extend...","Strengthens legs, core, and shoulders; improve...",Activates the solar plexus; enhances determina...,Warrior I Pose or Warrior II Pose,Warrior III Pose
5,Utthita Hasta Padangustasana,Extended Hand-To-Big-Toe Pose,"From Mountain Pose, extend one leg forward, ho...",Improves balance and flexibility; stretches ha...,Enhances focus and concentration; activates en...,Hand-to-Big-Toe Pose or Standing Forward Bend ...,Extended Hand-To-Big-Toe Pose
6,Yoganidrasana,Yogic Sleep Pose,"Lie on the back, hug the knees into the chest,...",Relaxes the entire body; reduces stress and an...,Induces deep relaxation and promotes a meditat...,Supine Twist Pose or Reclined Hand-to-Big-Toe ...,Yogic Sleep Pose
7,Supta Padangusthasana,Reclining Hand-To-Big-Toe Pose,"Lie on the back, lift one leg, and hold the bi...",Stretches hamstrings and calves; improves flex...,Calms the mind and releases tension,Reclined Hand-to-Big-Toe Pose or Supine Leg Ex...,Reclining Hand-To-Big-Toe Pose
8,Padangusthasana,Big Toe Pose,"From Mountain Pose, lift one leg, hold the big...",Stretches hamstrings and calves; improves flex...,Grounds energy and improves focus; enhances fl...,Seated Hand-to-Big-Toe Pose or Reclined Hand-t...,Big Toe Pose
9,Uttanasana,Standing Forward Bend,"Stand with feet hip-width apart, hinge at the ...",Stretches hamstrings and lower back; improves ...,Calms the mind and releases tension,Forward Fold with Bent Knees Pose,Standing Forward Bend Pose
10,Utthita Hasta Padangustasana,Extended Hand-To-Big-Toe Pose,"From Mountain Pose, extend one leg forward, ho...",Improves balance and flexibility; stretches ha...,Enhances focus and concentration; activates en...,Hand-to-Big-Toe Pose or Standing Forward Bend ...,Extended Hand-To-Big-Toe Pose


# Plan based on body parts and transition based on difficulty level

In [ ]:
def flatten_list(xss):
  return [
    x
    for xs in xss
    for x in xs
]

In [ ]:
def find_not_included(thing, ex_lst):
  return thing if thing not in ex_lst else ""

In [ ]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

In [ ]:
for b in body_parts:
  print(b, find_poses_of_x_benefits(b).shape)

leg (21, 13)
hip (38, 13)
shoulder (48, 13)
chest (27, 13)
neck (4, 13)
back (15, 13)
arm (28, 13)
front (2, 13)
core (34, 13)
hamstring (17, 13)
ankle (3, 13)
wrist (3, 13)
foot (0, 13)
knee (0, 13)
hand (0, 13)
head (0, 13)


In [ ]:
#@title
def sequence_df_by_body_part_no_dup(bp):
  shoulder_df = find_poses_of_x_benefits(bp)[simple_cols]
  intros = flatten_list(shoulder_df["Intro Poses"].apply(lambda x: x.split(" or ")).values)
  sequence = shoulder_df[shoulder_df["Intro Poses"] == ""].Pose.values.tolist()
  for i in intros:
    # if i in sequence:
      sequence += [find_not_included(add_end_pose(i), sequence)]
    # print(i, len(sequence), find_poses_of_x_benefits(i, "Intro Poses")["English"].values)
      sequence += [find_not_included(add_end_pose(x), sequence) for x in find_poses_of_x_benefits(i, "Intro Poses")["English"].values if add_end_pose(x) not in sequence]
    # else:
    #   sequence += list(find_poses_of_x_benefits(i, "Intro Poses")["English"].values)
  sequence = [x for x in sequence if x != ""]
  sequence_df = pd.DataFrame(sequence, columns = ["Pose"])
  return sequence_df.drop_duplicates().merge(
    df, on = "Pose", how = "inner")

In [ ]:
def sequence_df_by_body_part_w_dup(bp):
  shoulder_df = find_poses_of_x_benefits(bp)[simple_cols]
  fl = flatten_list(shoulder_df["Intro Poses"].apply(lambda x: x.split(" or ")).values)
  intros = f7(fl)
  sequence = shoulder_df[shoulder_df["Intro Poses"] == ""].Pose.values.tolist()
  for i in intros:
    # if i in sequence:
      sequence += [add_end_pose(i)]
    # print(i, len(sequence), find_poses_of_x_benefits(i, "Intro Poses")["English"].values)
      sequence += [add_end_pose(x) for x in find_poses_of_x_benefits(i, "Intro Poses")["English"]]
    # else:
    #   sequence += list(find_poses_of_x_benefits(i, "Intro Poses")["English"].values)
  sequence = [x for x in sequence if x != ""]
  sequence_df = pd.DataFrame(sequence, columns = ["Pose"])
  return sequence_df.merge(
    df, on = "Pose", how = "left").dropna()

In [ ]:
#@title shoulder sequence
shoulder_df = find_poses_of_x_benefits("shoulder")[simple_cols]

In [ ]:
intros = flatten_list(shoulder_df["Intro Poses"].apply(lambda x: x.split(" or ")).values)

In [ ]:
len(set(intros)), shoulder_df.shape[0] + len(set(intros))

(54, 100)

In [ ]:
sequence = shoulder_df[shoulder_df["Intro Poses"] == ""].Pose.values.tolist()
for i in intros:
  # if i in sequence:
    sequence += [find_not_included(add_end_pose(i), sequence)]
  # print(i, len(sequence), find_poses_of_x_benefits(i, "Intro Poses")["English"].values)
    sequence += [find_not_included(add_end_pose(x), sequence) for x in find_poses_of_x_benefits(i, "Intro Poses")["English"].values if add_end_pose(x) not in sequence]
  # else:
  #   sequence += list(find_poses_of_x_benefits(i, "Intro Poses")["English"].values)
sequence = [x for x in sequence if x != ""]
# advanced = []
  # if i in advanced:
  #   sequence += [find_not_included(i, sequence)]
  #   advanced.remove(i)
  # else:
  #   sequence += [find_not_included(i, sequence)]
  #   advanced += [find_not_included(x, advanced) for x in find_poses_of_x_benefits(i, "Intro Poses")["English"].values if x not in advanced]

In [ ]:
sequence_df = pd.DataFrame(sequence, columns = ["Pose"])

In [ ]:
sequence_df.drop_duplicates().merge(
    df, on = "Pose", how = "inner"
)[simple_cols]

NameError: name 'sequence_df' is not defined

In [ ]:
sequence_df_by_body_part_w_dup("shoulder")[simple_cols]

<ipython-input-6-57fa8158c2b8>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return df[df[col].str.contains(x)]


,Sanskrit,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
1,Ashtanga Namaskara,Eight-Limbed Salutation,"From plank, lower the knees, chest, and chin t...","Strengthens arms, shoulders, and chest; tones ...",Builds strength and prepares the body for back...,Knees-Chest-Chin Pose or Modified Chaturanga Pose,Eight-Limbed Salutation Pose
3,Ashtanga Namaskara,Eight-Limbed Salutation,"From plank, lower the knees, chest, and chin t...","Strengthens arms, shoulders, and chest; tones ...",Builds strength and prepares the body for back...,Knees-Chest-Chin Pose or Modified Chaturanga Pose,Eight-Limbed Salutation Pose
4,Setu Bandha Sarvangasana,Bridge Pose,"Lie on the back, bend the knees, place the fee...","Strengthens legs, glutes, and lower back; open...",Activates the throat chakra; promotes self-exp...,Supine Hand-To-Big-Toe Pose,Bridge Pose
5,Camatkarasana,Wild Thing,"From Downward-Facing Dog, lift one leg, bend t...",Opens chest and shoulders; strengthens arms,Releases tension and promotes a sense of joy,Bridge Pose,Wild Thing Pose
6,Dhanurasana,Bow Pose,"Lie on the stomach, bend the knees, reach back...",Stretches front of the body; strengthens back ...,Activates the energy in the spine and invigora...,Locust Pose or Bridge Pose,Bow Pose
...,...,...,...,...,...,...,...
156,Supta Matsyendrasana,Supine Spinal Twist Pose,"Lie on the back, bend one knee, and place the ...","Stretches spine, shoulders, and hips; improves...",Activates energy in the solar plexus; enhances...,Reclined Twist Pose or Supine Twist Pose,Supine Spinal Twist Pose
158,Yoganidrasana,Yogic Sleep Pose,"Lie on the back, hug the knees into the chest,...",Relaxes the entire body; reduces stress and an...,Induces deep relaxation and promotes a meditat...,Supine Twist Pose or Reclined Hand-to-Big-Toe ...,Yogic Sleep Pose
159,Supta Matsyendrasana,Supine Spinal Twist Pose,"Lie on the back, bend one knee, and place the ...","Stretches spine, shoulders, and hips; improves...",Activates energy in the solar plexus; enhances...,Reclined Twist Pose or Supine Twist Pose,Supine Spinal Twist Pose
160,Dhanurasana,Bow Pose,"Lie on the stomach, bend the knees, reach back...",Stretches front of the body; strengthens back ...,Activates the energy in the spine and invigora...,Locust Pose or Bridge Pose,Bow Pose


In [ ]:
#@title hip
# Note Warrior II is said to be an intro to Warrior I
sequence_df_by_body_part_no_dup("hip")[simple_cols]

,Hindi,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
0,Virabhadrasana II,Warrior II,"From Mountain Pose, step one foot back and ben...",Strengthens legs and opens hips; stretches groins,Builds strength and stability; enhances determ...,Warrior I Pose or Extended Side Angle Pose,Warrior II Pose
1,Virabhadrasana I,Warrior I,"From Mountain Pose, step one foot back and ben...","Strengthens legs, opens hips and chest",Energizes and focuses the mind,Warrior II Pose,Warrior I Pose
2,Utthita Parsvakonasana,Extended Side Angle Pose,"From Warrior II, extend the front arm forward,...","Strengthens legs, hips, and core; stretches si...",Energizes and balances the body; enhances stamina,Warrior II Pose,Extended Side Angle Pose
3,Virabhadrasana III,Warrior III,"From Mountain Pose, step one foot back, extend...","Strengthens legs, core, and shoulders; improve...",Activates the solar plexus; enhances determina...,Warrior I Pose or Warrior II Pose,Warrior III Pose
4,Paschimottanasana,Seated Forward Bend,"Sit with legs extended, hinge at the hips, and...",Stretches hamstrings and lower back; improves ...,Calms the mind and releases tension,Staff Pose,Seated Forward Bend Pose
5,Janu Sirsasana,Head-to-Knee Forward Bend,Sit with one leg extended and the other foot a...,"Stretches hamstrings, opens hips, and stimulat...",Calms the mind and relieves stress,Seated Forward Bend Pose,Head-to-Knee Forward Bend Pose
6,Dandasana,Staff Pose,"Sit with legs extended, feet flexed, and hands...",Strengthens back and core; improves posture,Grounds energy and promotes stability; enhance...,Seated Forward Bend Pose,Staff Pose
7,Upavistha Konasana,Seated Wide-Legged Forward Bend,"Sit with legs spread wide, hinge at the hips, ...",Stretches inner thighs and groins; improves fl...,Balances energy in the lower chakras; enhances...,Wide-Angle Seated Forward Bend Pose,Seated Wide-Legged Forward Bend Pose
8,Gomukhasana,Cow Face Pose,"Sit with legs crossed, stack one knee on top o...",Stretches shoulders and hips; improves posture,Balances the nervous system and enhances conce...,Eagle Arms Pose,Cow Face Pose
9,Sukhasana,Easy Pose,"Sit with legs crossed, keeping the spine strai...",Improves posture and alignment; opens hips and...,Promotes relaxation and mental clarity,Seated Cross-Legged Pose,Easy Pose


In [ ]:
sequence_df_by_body_part_w_dup("hip")[simple_cols]

,Hindi,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
0,Virabhadrasana II,Warrior II,"From Mountain Pose, step one foot back and ben...",Strengthens legs and opens hips; stretches groins,Builds strength and stability; enhances determ...,Warrior I Pose or Extended Side Angle Pose,Warrior II Pose
1,Virabhadrasana I,Warrior I,"From Mountain Pose, step one foot back and ben...","Strengthens legs, opens hips and chest",Energizes and focuses the mind,Warrior II Pose,Warrior I Pose
2,Utthita Parsvakonasana,Extended Side Angle Pose,"From Warrior II, extend the front arm forward,...","Strengthens legs, hips, and core; stretches si...",Energizes and balances the body; enhances stamina,Warrior II Pose,Extended Side Angle Pose
3,Virabhadrasana III,Warrior III,"From Mountain Pose, step one foot back, extend...","Strengthens legs, core, and shoulders; improve...",Activates the solar plexus; enhances determina...,Warrior I Pose or Warrior II Pose,Warrior III Pose
4,Paschimottanasana,Seated Forward Bend,"Sit with legs extended, hinge at the hips, and...",Stretches hamstrings and lower back; improves ...,Calms the mind and releases tension,Staff Pose,Seated Forward Bend Pose
...,...,...,...,...,...,...,...
114,Garbha Pindasana,Embryo in Womb Pose,"From Lotus Pose, lift the legs, bend the knees...","Stretches spine, shoulders, and hips; improves...",Activates energy in the solar plexus; enhances...,Child's Pose or Happy Baby Pose,Embryo in Womb Pose
116,Sukhasana,Easy Pose,"Sit with legs crossed, keeping the spine strai...",Improves posture and alignment; opens hips and...,Promotes relaxation and mental clarity,Seated Cross-Legged Pose,Easy Pose
118,Supta Matsyendrasana,Supine Spinal Twist Pose,"Lie on the back, bend one knee, and place the ...","Stretches spine, shoulders, and hips; improves...",Activates energy in the solar plexus; enhances...,Reclined Twist Pose or Supine Twist Pose,Supine Spinal Twist Pose
120,Yoganidrasana,Yogic Sleep Pose,"Lie on the back, hug the knees into the chest,...",Relaxes the entire body; reduces stress and an...,Induces deep relaxation and promotes a meditat...,Supine Twist Pose or Reclined Hand-to-Big-Toe ...,Yogic Sleep Pose


In [ ]:
#@title core
# Note Warrior II is said to be an intro to Warrior I
sequence_df_by_body_part_no_dup("core")[simple_cols]

,Hindi,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
0,Phalakasana,Plank Pose,"From Plank Pose, engage the core, and straight...","Strengthens arms, shoulders, and core; improve...",Builds heat and energy in the body; enhances d...,Tabletop Pose,Plank Pose
1,Chaturanga Dandasana,Four-Limbed Staff Pose,"Start in plank position, lower the body down w...","Strengthens arms, wrists, and core; tones abdo...",Builds heat and enhances endurance,Plank Pose or Modified Push-up Pose,Four-Limbed Staff Pose
2,Pincha Mayurasana,Forearm Stand,"From Downward-Facing Dog, walk the feet in, li...","Strengthens arms, shoulders, and core; improve...",Invigorates the body and mind; stimulates the ...,Dolphin Pose or Forearm Plank Pose,Forearm Stand Pose
3,Vasisthasana,Side Plank Pose,"From Plank Pose, shift weight to one hand, sta...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Modified Side Plank Pose,Side Plank Pose
4,Dwi Pada Viparita Dandasana,Two-Legged Inverted Staff Pose,"Lie on the back, lift the legs overhead, and s...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Inverted Staff Pose or Inverted Plank Pose,Two-Legged Inverted Staff Pose
5,Ardha Pincha Mayurasana,Dolphin Pose,"From Downward-Facing Dog, walk the feet in, li...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Forearm Plank Pose,Dolphin Pose
6,Parsva Bakasana,Side Crow Pose,"From Crow Pose, shift weight to one side, twis...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Crow Pose or Side Plank Pose with Knee Tuck,Side Crow Pose
7,Mayurasana,Peacock Pose,"Kneel with knees wide, place the hands on the ...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Plank Pose with Forearms on the Ground,Peacock Pose
8,Astavakrasana,Eight-Angle Pose,"Sit with legs extended, bend one knee and plac...","Strengthens wrists, arms, and core; improves b...",Develops focus and concentration; activates th...,Seated Twist Pose or Half Lord of the Fishes Pose,Eight-Angle Pose
9,Eka Pada Koundinyanasana II,One-Legged Sage Koundinya's Pose II,"From seated, extend one leg, bend the other kn...","Strengthens arms, core, and legs; improves bal...",Balances energy in the body; promotes focus an...,Seated Twist Pose or Half Lord of the Fishes Pose,One-Legged Sage Koundinya's Pose II


In [ ]:
sequence_df_by_body_part_w_dup("core")[simple_cols]

,Hindi,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
0,Phalakasana,Plank Pose,"From Plank Pose, engage the core, and straight...","Strengthens arms, shoulders, and core; improve...",Builds heat and energy in the body; enhances d...,Tabletop Pose,Plank Pose
1,Chaturanga Dandasana,Four-Limbed Staff Pose,"Start in plank position, lower the body down w...","Strengthens arms, wrists, and core; tones abdo...",Builds heat and enhances endurance,Plank Pose or Modified Push-up Pose,Four-Limbed Staff Pose
2,Pincha Mayurasana,Forearm Stand,"From Downward-Facing Dog, walk the feet in, li...","Strengthens arms, shoulders, and core; improve...",Invigorates the body and mind; stimulates the ...,Dolphin Pose or Forearm Plank Pose,Forearm Stand Pose
3,Vasisthasana,Side Plank Pose,"From Plank Pose, shift weight to one hand, sta...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Modified Side Plank Pose,Side Plank Pose
4,Dwi Pada Viparita Dandasana,Two-Legged Inverted Staff Pose,"Lie on the back, lift the legs overhead, and s...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Inverted Staff Pose or Inverted Plank Pose,Two-Legged Inverted Staff Pose
...,...,...,...,...,...,...,...
121,Anantasana,Vishnu's Couch Pose,"Lie on the side, lift the top leg, hold the bi...","Strengthens legs, hips, and core; improves bal...",Activates energy in the solar plexus; enhances...,Reclining Hand-To-Big-Toe Pose or Reclining Bo...,Vishnu's Couch Pose
123,Parsva Bakasana,Side Crow Pose,"From Crow Pose, shift weight to one side, twis...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Crow Pose or Side Plank Pose with Knee Tuck,Side Crow Pose
125,Mayurasana,Peacock Pose,"Kneel with knees wide, place the hands on the ...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Plank Pose with Forearms on the Ground,Peacock Pose
127,Tulasana,Scale Pose,"Sit with knees bent and feet on the floor, lif...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Boat Pose with Bent Knees,Scale Pose


In [ ]:
#@title leg
# Note Warrior II is said to be an intro to Warrior I
from google.colab import data_table
data_table.DataTable(
    sequence_df_by_body_part_w_dup("leg")[simple_cols][["English", "Instruction"]],
    include_index=False, num_rows_per_page=200)

<ipython-input-35-57fa8158c2b8>:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  return df[df[col].str.contains(x)]


,English,Instruction
0,Mountain Pose,"Stand with feet together, arms by the sides, a..."
1,Warrior II,"From Mountain Pose, step one foot back and ben..."
2,Warrior I,"From Mountain Pose, step one foot back and ben..."
3,Extended Side Angle Pose,"From Warrior II, extend the front arm forward,..."
4,Warrior III,"From Mountain Pose, step one foot back, extend..."
...,...,...
191,Forearm Stand,"From Downward-Facing Dog, walk the feet in, li..."
192,Handstand,"From Downward-Facing Dog, walk the feet in and..."
193,Dolphin Plank Pose,"From Dolphin Pose, lower the chest towards the..."
194,Supported Headstand,"From a kneeling position, place the forearms o..."


In [ ]:
#@title arm
# Note Warrior II is said to be an intro to Warrior I
sequence_df_by_body_part_w_dup("arm")[simple_cols]

,Hindi,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
0,Phalakasana,Plank Pose,"From Plank Pose, engage the core, and straight...","Strengthens arms, shoulders, and core; improve...",Builds heat and energy in the body; enhances d...,Tabletop Pose,Plank Pose
1,Chaturanga Dandasana,Four-Limbed Staff Pose,"Start in plank position, lower the body down w...","Strengthens arms, wrists, and core; tones abdo...",Builds heat and enhances endurance,Plank Pose or Modified Push-up Pose,Four-Limbed Staff Pose
2,Pincha Mayurasana,Forearm Stand,"From Downward-Facing Dog, walk the feet in, li...","Strengthens arms, shoulders, and core; improve...",Invigorates the body and mind; stimulates the ...,Dolphin Pose or Forearm Plank Pose,Forearm Stand Pose
3,Vasisthasana,Side Plank Pose,"From Plank Pose, shift weight to one hand, sta...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Modified Side Plank Pose,Side Plank Pose
4,Dwi Pada Viparita Dandasana,Two-Legged Inverted Staff Pose,"Lie on the back, lift the legs overhead, and s...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Inverted Staff Pose or Inverted Plank Pose,Two-Legged Inverted Staff Pose
...,...,...,...,...,...,...,...
99,Dwi Pada Viparita Dandasana,Two-Legged Inverted Staff Pose,"Lie on the back, lift the legs overhead, and s...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Inverted Staff Pose or Inverted Plank Pose,Two-Legged Inverted Staff Pose
101,Parsva Bakasana,Side Crow Pose,"From Crow Pose, shift weight to one side, twis...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Crow Pose or Side Plank Pose with Knee Tuck,Side Crow Pose
103,Mayurasana,Peacock Pose,"Kneel with knees wide, place the hands on the ...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Plank Pose with Forearms on the Ground,Peacock Pose
105,Tulasana,Scale Pose,"Sit with knees bent and feet on the floor, lif...","Strengthens arms, shoulders, and core; improve...",Activates the solar plexus; enhances determina...,Boat Pose with Bent Knees,Scale Pose


In [ ]:
#@title ankle
# Note Warrior II is said to be an intro to Warrior I
sequence_df_by_body_part_w_dup("ankle")[simple_cols]

,Hindi,English,Instruction,Anatomic Benefits,Energetic Benefits,Intro Poses,Pose
1,Malasana,Garland Pose,"Squat with feet wider than hip-width apart, lo...",Opens hips and groins; stretches ankles and lo...,Connects with the root chakra; promotes a sens...,Deep Squat Pose,Garland Pose
3,Supta Virasana,Reclining Hero Pose,"Kneel with the buttocks on the heels, lower th...",Stretches thighs and abdomen; opens chest,Connects with the heart chakra; promotes a sen...,Reclined Hero Pose or Supported Bridge Pose,Reclining Hero Pose
4,Virasana,Hero Pose,"Kneel with the buttocks on the heels, sit back...",Stretches thighs and ankles; improves posture,Grounds energy and promotes stability; enhance...,Reclined Hero Pose,Hero Pose


# Write out

In [ ]:
write_out = False # @param {type:"boolean"}

In [ ]:
if write_out:

  from google.colab import auth
  auth.authenticate_user()

  import gspread
  from google.auth import default
  creds, _ = default()

  # gc = gspread.authorize(creds)
  sh = gc.create('A new spreadsheet')

  # Open our new sheet and add some data.
  worksheet = gc.open(SHEET_NAME)[sh]

  cell_list = worksheet.range('A1:')

  worksheet.update_cells(cell_list)
  # Go to https://sheets.google.com to see your new spreadsheet.